# Análisis de los datos de pacientes covid

[/] Navegar expedientes (helpers.py)  
[AC] Obtener datos de todos los pacientes (extraerDatosPacientes.py y scrapperHTML.py > datos_pacientes.pk)  
[/] Remover nombres, tratante y fechas de nacimiento de todos los labs pdf (replaceStringsPDF.py)  
[AA] Extraer tablas de pdf (scrapperPDF.py)  
[ME] Anonimizado agresivo sobre notas de html (anonimizado.py)  
[AC] Covidminer sobre notas de html  